# Signal processing

In [ ]:
import json
import urllib

import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import column, layout, row, gridplot
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure
from scipy import fft, signal

import matplotlib.pyplot as plt

output_notebook()

temp_path = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/current_clamp/"
with urllib.request.urlopen(temp_path + "11.json") as url:
    temp = json.load(url)
    data = np.array(temp["array"])

temp_path = (
    "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/mepsc/"
)
with urllib.request.urlopen(temp_path + "1.json") as url:
    temp = json.load(url)
    mepsc = np.array(temp["array"])

In this chapter we will cover what I consider core signal processing techniques such as convolution, filtering, FFT, peak finding, derivatives and integrals.

## Derivatives and integrals
We will start with derivatives and integrals. Derivatives and integrals are some of the simplest signal processing that we can do. While derivatives and integrals may seem like intimidating calculus they are actually pretty simple for discrete data. 

### Derivatives
#### How to take a derivative
If you remember from calculus that the derivative is defined by: $$L=\lim _{h\to 0}{\frac {f(a+h)-f(a)}{h}}$$ This means that we take the slope between each set of values. For continuous data that means there are an infinite number of differences but, for discrete data there are discrete points which means that we can just take the difference between neighboring points for our x and y data. Numpy has a built in function to do this called `np.diff`. If you have evenly spaced data as is for most data that we collect in the digital realm you dy is technically just `y[1]-y[0]`. If you want to get fancy you can use some called forward, central, or backward differences. In numpy there is a func called `np.gradient` that calculates the second order difference, which if you have evenly spaced data is just this equation: $$\hat f_{i}^{(1)}=\frac{f\left(x_{i+1}\right) - f\left(x_{i-1}\right)}{2h}+ \mathcal{O}\left(h^{2}\right)$$ The second order difference just means that you grab the points to each side of the current index and find the slope between those. The gradient is nice because you end up with an output that is the same size as the input. The drawback is that if you have a signal with small peaks you will lose a lot of the variability in your signal. One way around that is to upsample your signal.
#### When is a derivative useful?
The derivative has many uses. The derivative of voltage signals can be used to transform it into current signals if you are recording voltage from a capacitor (a cell is a capacitor). You can use derivatives to detrend signals. Many statistical algorithms used to analyze signals assume something called stationarity which means that a signal has a relatively constant mean and variance over time. The derivative is a great way to create a stationary signal. We can use the derivative to find specific features of a signal like in the [current clamp](#cc_pt1) chapter. You could correlate the derivatives instead the actual signal this will help find where there are overlapping rates of change.

In [ ]:
dy = np.diff(data)
dx = np.diff(np.arange(data.size) / 10)
derivative = (dy / dx)[2500:10500]
fig1 = figure(height=250, width=350)
fig1.line(np.arange(derivative.size), derivative)
fig2 = figure(height=250, width=350, x_range=fig1.x_range, y_range=fig1.y_range)
gradient = np.gradient(data, np.arange(data.size) / 10)[2500:10500]
fig2.line(np.arange(gradient.size), gradient)
show(gridplot([[fig1, fig2]]))

## Convolution
Convolution is one the most used signal processing algorithms. It is used in filtering, template matching (which is a type of filtering), spike cross-correlation (different from Pearson's correlation) and is the core of convolution neural networks. We are going to focus on 1D convolution. If you have gone through the mspc chapter you have seen template matching (a version of convolution) used to find PCS. 

The basic idea of convolution is you are weighting an input array with a template and sliding that template over your input array. If you want to do correlation you just reverse your template. 

### Time domain convolution
Below you can see an implementation of time domain convolution in Python. What happens is that for each value in one array you multiply all the values in the second array and add it to an output array. Time domain convolution outputs an array whose length equals `len(input) + len(template) - 1`. The additive length of your output means your signal is shifted in time by len(template)/2. There are several ways you can output time domain convolution. You can output the arrays only where there is 100% overlap overlap between the two. You can output the full output like I have done below. You can output the same length. For most electrophysiological signal processing we want the same length output. However, there are several ways you can output the same length. One is to get the "zero phase" output by subset the array like this `output[template.size//2:input.size+template.size//2]` or you can just grab

### Frequency domain convolution
For frequency domain convolution you need to run the forward FFT on both of your arrays. You will need to zero-pad your shortest array to the same length as the longer array. Once you have the forward FFT of both signals you just do element-wise multiplication then you take the resulting array and do the backward FFT. One thing to note is that if your input signals are real you can run a faster version of the FFT. The FFT convolution will output an array of length equal to the longest array even if you zero-pad your signal. 

Below you can see a comparison of the two by convolving an array of noise with a gaussian curve (a common way to filter a signal).

In [ ]:
def time_convolution(array1, array2):
    output = np.zeros(array1.size + array2.size - 1)
    for i in range(array1.size):
        for j in range(array2.size):
            output[i + j] += array1[i] * array2[j]
    return output


def fft_convolution(array1, array2):
    size = fft.next_fast_len(max(array1.size, array2.size))
    output = fft.irfft(fft.rfft(array1, n=size) * fft.rfft(array2, n=size))
    return output

In [ ]:
rng = np.random.default_rng(42)
array1 = rng.random(1000)
array1 -= array1.mean()
array2 = signal.windows.gaussian(200, 3)
tconv = time_convolution(array1, array2)
fconv = fft_convolution(array1, array2)
fig, ax = plt.subplots(nrows=2, ncols=2, constrained_layout=True)
ax = ax.flat
ax[0].plot(array1, linewidth=1)
ax[0].set_title("Original")
ax[1].plot(array2, linewidth=1)
ax[1].set_title("Gaussian kernel")
ax[2].plot(tconv, linewidth=1)
ax[2].set_title("Time Convolution")
ax[3].plot(fconv, linewidth=1)
ax[3].set_title("FFT Convolution")
for i in ax:
    i.grid(which="both", alpha=0.5)

You may notice that the original signal looks fuzzier than than the convolved signal. This is because the gaussian kernel acts as a simple high pass filter.

## Filtering
Filtering comes in a couple types: IIR filters, FIR filters, convolution filters and moving window filters. When we talk about designing filters, you may come across 4 types of filters: lowpass, highpass, bandpass and notch filters. Lowpass filters let low frequency signals pass and attenuate (reduce the gain/power) of high frequency signals. Highpass filters let high frequencies signals pass and attenuate low frequency signals. Bandpass filters attenuate both low and high frequencies but let frequenceis inbetween a set of frequencies pass such as 300-6000 Hz bandpass that can be used to isolate single unit spikes. Notch filters let most low and high frequency signals pass but typically attentuate a small frequency band such 58-62 Hz to remove line noise (at least in the US). The frequencies that filters let through unattenuated are considered the passband of the filter and the stopband is considered where the frequencies are fully attenuated by the filters (amount of attenuation depends on the filter). Most filters, particularly the ones we use, have a roll off which means that there is some set of frequencies that will be partially attenuated. Ripple is considered unwanted change in gain in the passband and stopband of a filter. Filters can also cause ripple in the time domain signal which means the filter is just introducing time-decaying oscillations into the signal which is usually unwanted. Filters with sharp cutoffs are more likely to have ripple and cause ringing.

### Convolution filters
Convolution filters are essentially convolving one array with another like we convolved the random noise data with a gaussian filter. Additionally, many FIR filters are just convolution filters. Simple convolution filters typically just filter out high frequency content.

### Moving window filters
Moving window filters are things like a sliding median or mean. Moving window filters typically act as lowpass filters. They are good for when your data is not sampled in the typical Hertz range but in days, hours, etc. For example you might use a moving mean to smooth stock price data or sales data. Moving window filters can shorten your data since your aggregating your data, however there are ways to make sure your signal is the same length. Moving window filters can also change the phase of your signal. The larger the window the more potential there is for phase change.

#### Moving mean filter

In [ ]:
rng = np.random.default_rng(42)
array1 = rng.random(500)
array1 -= array1.mean()
window_size = 10
mov_mean = np.zeros(array1.size)
for i in range(1, array1.size+1):
    j = i -1
    if j < window_size:
        mov_mean[j] = array1[:i].mean()
    else:
        mov_mean[j] = array1[i-10:i].mean()
fig, ax = plt.subplots()
ax.plot(array1, linewidth=1)
_ = ax.plot(mov_mean, linewidth=2)

### Infinite impulse response (IIR) and minimal-phase filters (primarily the Bessel and Butterworth filters)
IIR filters as their name implies have an impulse response that is infinitely long and have internal feedback. In Python they including digital versions of analog filters such as the Bessel and Butterworth filter. The Bessel and Butterworth filter are probably the most commonly used filters in electrophysiology. IIR filters are consider minimal-phase filters which means they disrupt the phase (oscillations) of the signal by introducing a frequency dependent delays in the signal. The Bessel and Butterworth filters have very little ripple or change in gain in frequencies due to the filter. Bessel and Butterworth filters basically are tradeoffs between ripple, the steepness of the filter cutoff and the phase delay. When designing these filters in Python we generally just need to supply the what type of passband we want (lowpass, highpass, bandpass) and the order of the filter. The order of the filter is the steepness of the cutoff between the passband and stopband. In Python there are several different computation ways to create and use these filters. You can use an A/B filter, SOS filter or create an impulse response and convolve the filter. Unless you need GPU acceleration I recommend the SOS filter since it is numerically stable compared the A/B filter and impulse response filter. While Bessel and Butterworth filters change the phase of your signal, in the digital realm we can filter the signal in two directions which removes the phase changes but filters your data twice.

#### Frequency response of different Butterworth filters

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8, 3), constrained_layout=True)
fig.suptitle("Effect of order on gain")
ax[0].set_title("Lowpass")
ax[1].set_title("Highpass")
for i in ax:
    i.grid(which="both", axis="both")
    i.set_xlabel("Frequency [rad/s]")
    i.set_ylabel("Amplitude [dB]")
for j in [
    (4, 600, "lowpass"),
    (8, 600, "lowpass"),
]:
    b, a = signal.butter(j[0], [j[1]], btype=j[2], analog=True)
    w, h = signal.freqs(b, a)
    w, h = signal.freqs(b, a)
    ax[0].semilogx(w, 20 * np.log10(abs(h)))
for j in [
    (4, 600, "highpass"),
    (8, 600, "highpass"),
]:
    b, a = signal.butter(j[0], [j[1]], btype=j[2], analog=True)
    w, h = signal.freqs(b, a)
    w, h = signal.freqs(b, a)
    ax[1].semilogx(w, 20 * np.log10(abs(h)))

#### Effect of filtering on phase using a Butterworth filter

In [ ]:
array1 = mepsc[2750:3250]
array1 -= array1.mean()
high = signal.butter(4, [600], btype="lowpass", output="sos", fs=10000)
single = signal.sosfilt(high, array1)
double = signal.sosfiltfilt(high, array1)
x = np.arange(single.size)
fig, ax = plt.subplots(constrained_layout=True)
ax.plot(x, array1, color="black", linewidth=1, alpha=0.2, label="Original")
ax.plot(x, single, color="orange", alpha=0.8, label="Minimal phase")
ax.plot(x, double, color="magenta", alpha=0.8, label="Zero-phase")
_ = ax.legend()

### Finite impulse response (FIR) and linear-phase filters
FIR filters are purely digital filters whose impulse response goes to zero in a finite amount of time. FIR filters are considered linear phase which means they do not affect the phase of the signal in passband but can have variable phase disruptions in the stopband. Linear phase response is extremely useful for telecommunications applications. The most common FIR filters are the sinc function window by a window such as the gaussian or Hann window. FIR filters have an order which is basically the length in samples of the filter. FIR filters are also known to cause ringing, however this can be minimized with longer filters and choice of window.